In [190]:
from keras.models import load_model
from pathlib import Path
import os
import librosa
import numpy as np
import pickle
from sklearn.metrics import classification_report
from sklearn import preprocessing

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [191]:
# parameters
duration = 2.97
sr = 22050 # if sampling rate is different, resample it to this
input_root = './../../../models/'
input_whole = input_root + 'cnn_dong_model_whole.h5'
input_label = input_root + 'label.pkl'
default_song = Path("../../../audio/testfiles/GTZAN/genres/pop/pop.00001.wav")
fmax = 10000 # maximum frequency considered
fft_window_points = 512
fft_window_dur = fft_window_points * 1.0 / sr # 23ms windows
hop_size = int(fft_window_points/ 2) # 50% overlap between consecutive frames
n_mels = 64

In [192]:
model = load_model(input_whole)

In [193]:
def load_song(song = default_song, offset = 0):
    offset = duration*offset
    y, sr = librosa.load(song, mono=True, offset=offset, duration=duration)
    m_sp = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=fft_window_points,
                                              hop_length=hop_size, n_mels=n_mels,
                                              fmax=fmax)
    m_sp = np.expand_dims(m_sp, 0)
    m_sp = np.expand_dims(m_sp, 3)
    return m_sp

In [194]:
with open(input_label, 'rb') as f:
    lb = pickle.load(f)

In [195]:
song = load_song()
#np.expand_dims(song, axis=1)
#song.append([1])
print(type(song))
print(np.shape(song))
prediction = model.predict(song)
print(prediction)
print(prediction.argmax(axis=-1))

<class 'numpy.ndarray'>
(1, 64, 256, 1)
[[1.9037491e-03 1.9418549e-09 6.2142976e-07 3.1947952e-02 1.7766736e-01
  6.6977037e-09 9.7064622e-05 7.8751349e-01 7.4354600e-04 1.2622590e-04]]
[7]


In [196]:
predictions = model.predict(song)
print(lb.classes_[prediction.argmax(axis=-1)])

['pop']
